In [13]:
import os
from imageio import imread
from pathlib import Path
import numpy as np

In [14]:
from tensorflow.keras import datasets
from tensorflow.keras.utils import to_categorical

(train_images_org, train_labels), (test_images_org, test_labels) = datasets.cifar10.load_data()
train_images_img, test_images_img = train_images_org.astype(dtype=np.complex64) / 255.0, test_images_org.astype(dtype=np.complex64) / 255.0
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

In [15]:
from tensorflow.keras import models
import tensorflow.keras.layers as layers

def get_tf_sequential_model():
    model = models.Sequential()
    model.add(layers.Input(shape=(32, 32, 3)))   # Always use ComplexInput at the start
    model.add(layers.Conv2D(32, (3, 3), activation="relu"))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation="relu"))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation="relu"))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(10, activation='softmax'))
    return model

get_tf_sequential_model().summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 15, 15, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 4, 4, 64)          36928     
                                                                 
 flatten_2 (Flatten)         (None, 1024)             

In [16]:
import cvnn.layers as complex_layers

def get_cvnn_sequential_model():
    model = models.Sequential()
    model.add(complex_layers.ComplexInput(input_shape=(32, 32, 3)))   # Always use ComplexInput at the start
    model.add(complex_layers.ComplexConv2D(32, (3, 3), activation="cart_relu"))
    model.add(complex_layers.ComplexMaxPooling2D((2, 2)))
    model.add(complex_layers.ComplexConv2D(64, (3, 3), activation="cart_relu"))
    model.add(complex_layers.ComplexMaxPooling2D((2, 2)))
    model.add(complex_layers.ComplexConv2D(64, (3, 3), activation="cart_relu"))
    model.add(complex_layers.ComplexFlatten())
    model.add(complex_layers.ComplexDense(64, activation="cart_relu"))
    model.add(complex_layers.ComplexDense(10, activation='softmax_real_with_abs'))
    return model

get_cvnn_sequential_model().summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 complex_conv2d_9 (ComplexCo  (None, 30, 30, 32)       1792      
 nv2D)                                                           
                                                                 
 complex_max_pooling2d_6 (Co  (None, 15, 15, 32)       0         
 mplexMaxPooling2D)                                              
                                                                 
 complex_conv2d_10 (ComplexC  (None, 13, 13, 64)       36992     
 onv2D)                                                          
                                                                 
 complex_max_pooling2d_7 (Co  (None, 6, 6, 64)         0         
 mplexMaxPooling2D)                                              
                                                                 
 complex_conv2d_11 (ComplexC  (None, 4, 4, 64)        

#### Adding a imaginary component into the real data and running the value and using CVNN

In [17]:
#Running the CVNN model on the data

from tensorflow import losses, metrics

model_cvnn = get_cvnn_sequential_model()
model_cvnn.compile(optimizer='sgd', loss='categorical_crossentropy', metrics='accuracy')
    
history = model_cvnn.fit(train_images_img, train_labels, epochs=100, validation_data=(test_images_img, test_labels),batch_size=32)

Epoch 1/100
  93/1563 [>.............................] - ETA: 3:49 - loss: 2.2888 - accuracy: 0.1374

KeyboardInterrupt: 

In [ ]:
#Running an ANN on the data while ignoring the imaginary values

model_ann = get_tf_sequential_model()
model_ann.compile(optimizer='sgd', loss='categorical_crossentropy', metrics='accuracy')
    
history = model_ann.fit(train_images_img, train_labels, epochs=100, validation_data=(test_images_img, test_labels),batch_size=32)

Epoch 1/100
  30/1563 [..............................] - ETA: 2:03 - loss: 2.3032 - accuracy: 0.1250

KeyboardInterrupt: 

#### Now mapping the data from the real plane to the complex plane using Hilberts transform

In [20]:
import scipy

train_images_hil, test_images_hil = scipy.signal.hilbert(train_images_org/255), scipy.signal.hilbert(test_images_org/255)

: 

: 